In [11]:
# import pandas as pd
# import numpy as np

# # Parameters for the dataset
# num_records = 100  # Total number of sales records
# num_products = 10  # Total number of unique products

# # Generate product IDs
# product_ids = [f'P{i+1}' for i in range(num_products)]

# # Generate a date range for sales (e.g., the last 30 days)
# sale_dates = pd.date_range(end=pd.Timestamp.today(), periods=num_records)

# # Generate current stock levels (randomly between 20 and 100 for each product)
# current_stock = np.random.randint(20, 100, size=num_products)

# # Generate sales data
# sales_data = {
#     'product_id': np.random.choice(product_ids, size=num_records),
#     'sale_date': np.random.choice(sale_dates, size=num_records),
#     'sale_quantity': np.random.randint(1, 10, size=num_records)  # Random quantity between 1 and 10
# }

# # Create a DataFrame
# df_sales = pd.DataFrame(sales_data)

# # Calculate current stock after sales
# df_sales['current_stock'] = df_sales['product_id'].map(lambda x: current_stock[int(x[1:])-1]) - df_sales['sale_quantity']

# # Display the DataFrame
# print(df_sales)


In [20]:
import pandas as pd
import numpy as np

# Assuming df_sales is the DataFrame we generated earlier
df_sales = pd.read_csv("C:\storage\Internship\Inventory\dummy_data\dummy_data\sales_data.csv")
df_sales.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Sale_ID      500 non-null    object
 1   Store_ID     500 non-null    int64 
 2   Product_ID   500 non-null    object
 3   Customer_ID  500 non-null    object
 4   Quantity     500 non-null    int64 
 5   Sale_Date    500 non-null    object
 6   Sale_Amount  500 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 27.5+ KB


In [28]:
df_sales['Sale_Date'] = pd.to_datetime(df_sales['Sale_Date'])

In [45]:
df_sales['Sale_Date'].max()

Timestamp('2024-12-30 00:00:00')

In [29]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Sale_ID      500 non-null    object        
 1   Store_ID     500 non-null    int64         
 2   Product_ID   500 non-null    object        
 3   Customer_ID  500 non-null    object        
 4   Quantity     500 non-null    int64         
 5   Sale_Date    500 non-null    datetime64[ns]
 6   Sale_Amount  500 non-null    int64         
dtypes: datetime64[ns](1), int64(3), object(3)
memory usage: 27.5+ KB


In [37]:


# Aggregate sales data by product and sale date
df_agg = df_sales.groupby(['Product_ID', 'Sale_Date']).agg({'Quantity': 'sum'}).reset_index()
# print(df_agg.head())

# Create a complete date range for each product
Product_IDs = df_sales['Product_ID'].unique()
# print(Product_IDs)
all_dates = pd.date_range(start=df_agg['Sale_Date'].min(), end=df_agg['Sale_Date'].max())
# print(all_dates)
# Create a new DataFrame with all combinations of product IDs and dates
product_date_index = pd.MultiIndex.from_product([Product_IDs, all_dates], names=['Product_ID', 'Sale_Date'])
# print(product_date_index)
df_complete = pd.DataFrame(index=product_date_index).reset_index()
print(df_complete)

# Merge with the aggregated sales data
df_forecast = pd.merge(df_complete, df_agg, on=['Product_ID', 'Sale_Date'], how='left').fillna(0)

# Display the forecast DataFrame
print(df_forecast.head())


     Product_ID  Sale_Date
0          P003 2023-01-01
1          P003 2023-01-02
2          P003 2023-01-03
3          P003 2023-01-04
4          P003 2023-01-05
...         ...        ...
2915       P001 2024-12-26
2916       P001 2024-12-27
2917       P001 2024-12-28
2918       P001 2024-12-29
2919       P001 2024-12-30

[2920 rows x 2 columns]
  Product_ID  Sale_Date  Quantity
0       P003 2023-01-01       1.0
1       P003 2023-01-02       0.0
2       P003 2023-01-03       0.0
3       P003 2023-01-04       3.0
4       P003 2023-01-05       5.0


In [38]:
# Calculate a moving average for each product
window_size = 7  # 7-day moving average
df_forecast['moving_average'] = df_forecast.groupby('Product_ID')['Quantity'].transform(lambda x: x.rolling(window=window_size, min_periods=1).mean())

# Display the forecast DataFrame with moving averages
print(df_forecast.head(15))


   Product_ID  Sale_Date  Quantity  moving_average
0        P003 2023-01-01       1.0        1.000000
1        P003 2023-01-02       0.0        0.500000
2        P003 2023-01-03       0.0        0.333333
3        P003 2023-01-04       3.0        1.000000
4        P003 2023-01-05       5.0        1.800000
5        P003 2023-01-06       0.0        1.500000
6        P003 2023-01-07       0.0        1.285714
7        P003 2023-01-08       0.0        1.142857
8        P003 2023-01-09       4.0        1.714286
9        P003 2023-01-10       1.0        1.857143
10       P003 2023-01-11       0.0        1.428571
11       P003 2023-01-12       0.0        0.714286
12       P003 2023-01-13       0.0        0.714286
13       P003 2023-01-14       0.0        0.714286
14       P003 2023-01-15       0.0        0.714286


In [47]:
import statsmodels.api as sm

# Function to perform ARIMA forecasting
def arima_forecast(product_data):
    product_data.set_index('Sale_Date', inplace=True)
    product_data = product_data.resample('D').sum()  # Resample to daily frequency
    product_data = product_data.fillna(0)  # Fill missing values with 0
    model = sm.tsa.ARIMA(product_data['Quantity'], order=(1, 1, 1))  # ARIMA(1, 1, 1)
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=30)  # Forecast the next 30 days
    return forecast

# Forecast for each product
forecast_results = {}
for product in Product_IDs:
    product_data = df_forecast[df_forecast['Product_ID'] == product]
    forecast_results[product] = arima_forecast(product_data)

# Display forecast results
for product, forecast in forecast_results.items():
    print(f"Forecast for {product}:")
    print(forecast.head())


Forecast for P003:
2024-12-31    0.478612
2025-01-01    0.499271
2025-01-02    0.500163
2025-01-03    0.500202
2025-01-04    0.500203
Freq: D, Name: predicted_mean, dtype: float64
Forecast for P004:
2024-12-31    0.507728
2025-01-01    0.489172
2025-01-02    0.489073
2025-01-03    0.489072
2025-01-04    0.489072
Freq: D, Name: predicted_mean, dtype: float64
Forecast for P002:
2024-12-31    0.537473
2025-01-01    0.545075
2025-01-02    0.545182
2025-01-03    0.545184
2025-01-04    0.545184
Freq: D, Name: predicted_mean, dtype: float64
Forecast for P001:
2024-12-31    0.584347
2025-01-01    0.562263
2025-01-02    0.563098
2025-01-03    0.563066
2025-01-04    0.563068
Freq: D, Name: predicted_mean, dtype: float64
